In [1]:
# for reading in data
import pandas as pd

# data preprocessing
from pvops.text.preprocess import text_remove_numbers_stopwords
from nltk import corpus

# machine learning pipeline: vectorizer, model definitions, and scoring
from pvops.text.classify import classification_deployer
from pvops.text.defaults import supervised_classifier_defs
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, make_scorer


### Read in data

First, we'll read in the data and take a look at it. We would like to identify inverter-related records, so our column of interest is `Asset`.

In [2]:
CM_df = pd.read_csv('example_data/example_ML_ticket_data_missing_assets.csv')
CM_df['Asset'].head(10)

0    Combiner
1         Pad
2    Facility
3    Inverter
4    Facility
5         NaN
6         NaN
7         NaN
8         NaN
9    Facility
Name: Asset, dtype: object

We can see that many records have a missing `Asset` field. Let's see how many records are missing it.

In [3]:
count_null = CM_df['Asset'].isnull().sum()
print(f'{count_null}/{len(CM_df)} records have a missing asset ({count_null/len(CM_df)*100 :.1f}%).')

15/100 records have a missing asset (15.0%).


We would like to leverage as much of the data as we can for later analysis, so it would help to find a way to gap-fill these records that are missing the asset. We can do so by leveraging the plain-text event description in the `GeneralDesc` column. Let's pull out the data of interest.

In [4]:
# pull out the Asset and CompletionDesc columns, and any rows with nulls in either column
CM_nonnull = CM_df[['Asset','CompletionDesc']].dropna()
CM_nonnull.head(5)

,Asset,CompletionDesc
0,Combiner,cb 1.18 was found to have contactor issue woul...
1,Pad,self resolved. techdispatched: no
2,Facility,"all module rows washed, waiting for final repo..."
3,Inverter,14 nov: we were alerted that e-c3-1 had faulte...
4,Facility,assessed condition filters all inverters. litt...


For good measure, we can also look to see how many of the non-null records have an inverter-related asset:

In [5]:
count_inverter = CM_nonnull['Asset'].str.contains('Inverter').sum()
print(f'{count_inverter}/{len(CM_nonnull)} records have an inverter-related asset ({count_inverter/len(CM_nonnull)*100 :.1f}%).')

27/85 records have an inverter-related asset (31.8%).


### Naive approach: keyword method

As a first approach, we can perform a simple keyword-based method. We make a prediction of the asset based on whether the word "inverter" is present in the description.

In [6]:
# check if the event description contains the word 'inverter'; this is our prediction
prediction = CM_nonnull['CompletionDesc'].str.lower().str.contains('inverter')
# check if the asset itself contains the word 'inverter'; this is the true label
true_label = CM_nonnull['Asset'].str.lower().str.contains('inverter')

naive_num_correct = (prediction == true_label).sum()
print(f'{naive_num_correct}/{len(CM_nonnull)} records predicted correctly from keyword search '
      + f'({naive_num_correct/len(CM_nonnull)*100 :.1f}%).')

65/85 records predicted correctly from keyword search (76.5%).


This already does decently well. Let's see if we can improve our results via a more robust machine learning approach.

### Prepare data for supervised classification

First, we need to clean our text data. We can use `nltk`'s english stopwords list to remove common words that won't have much bearing on the semantic meaning of the records. All we need to do is pass this into the `text_remove_numbers_stopwords()` function from `pvops.text.preprocess`, which will handle all of this for us.

From there, we can make everything lowercase, and we're basically done. 

In [7]:
# clean the event description
stopwords = corpus.stopwords.words('english')
CM_nonnull['CompletionDescCleaned'] = CM_nonnull.apply(lambda x: text_remove_numbers_stopwords(x['CompletionDesc'], stopwords),
                                                    axis='columns')

# set event description to be lowercase
CM_nonnull['CompletionDescLower'] = CM_nonnull['CompletionDescCleaned'].str.lower()

CM_nonnull.head(5)

,Asset,CompletionDesc,CompletionDescCleaned,CompletionDescLower
0,Combiner,cb 1.18 was found to have contactor issue woul...,cb found contactor issue would close contactor...,cb found contactor issue would close contactor...
1,Pad,self resolved. techdispatched: no,self resolved techdispatched,self resolved techdispatched
2,Facility,"all module rows washed, waiting for final repo...",module rows washed waiting final report sun power,module rows washed waiting final report sun power
3,Inverter,14 nov: we were alerted that e-c3-1 had faulte...,nov alerted e c faulted upon investigation not...,nov alerted e c faulted upon investigation not...
4,Facility,assessed condition filters all inverters. litt...,assessed condition filters inverters little cl...,assessed condition filters inverters little cl...


Finally, we would like to have our response be a binary value: whether the asset relates to inverters or not. We can use the `remappings_asset.csv` file to help us with this.

In [8]:
remappings_df = pd.read_csv('example_data/remappings_asset.csv')
remappings_df[-10:] # end of the dataframe is more clear on what it is doing

,in,out_
44,Point of Interconnection,other
45,Racking/Trackers,tracker
46,Rooftop PV System,other
47,Site,other
48,String,other
49,String Inverter,inverter
50,Subarray,other
51,Summary,other
52,Tracker control unit,tracker
53,Tracking System,tracker


All we need to do is put together a dictionary that will let us map the asset in our dataframe to a more general asset description.

In [9]:
remapping_dict = {row['in'].lower() : row['out_'].lower() for _, row in remappings_df.iterrows()}
CM_nonnull['SimpleAsset'] = CM_nonnull['Asset'].apply(lambda x : remapping_dict[x.lower()])

# define our x as the general event description, and y as whether the asset is related to inverters
x = CM_nonnull['CompletionDescLower']
y = CM_nonnull['SimpleAsset'] == 'inverter'

### Model training and selection

Now, it's time to begin the model training. First, we need to turn our `x` into numerical data, which we can do via tf-idf vectorization. We'd like our vectorization to include the entire corpus of our event descriptions, even those where the asset is null. This is so we can use our exact same pipeline to predict the missing asset labels in the final step.

In [10]:
# fit the tf-idf vectorizer on the whole corpus, including the rows w/o asset that we plan to gapfill using our model
all_descr = CM_df.dropna(subset='CompletionDesc')['CompletionDesc']
all_descr = all_descr.str.lower().astype('str')
vectorizer = TfidfVectorizer(min_df=1, stop_words=stopwords, ngram_range=(1,2), sublinear_tf=True)
vectorizer.fit(all_descr);

Now, we can fit models across various parameters using the `classification_deployer()` function from `pvops.text.classify`. All we need to do is define our pipeline, our models, our parameters, and our scoring function.

The pipeline and scoring function are easy enough. We can get a good set of model parameters to search over from `pvops.text.defaults`, as shown below:

In [11]:
# first do vectorization, then classification
pipeline_steps = [('tfidf', vectorizer),
                  ('clf', None)]

# basic accuracy score metric
scorer = make_scorer(accuracy_score)

# models that we will use
model_names = ['LinearSVC', 'LogisticRegression', 'PassiveAggressiveClassifier']

# get the default parameter grid from pvops for these models
(params, model_instances) = supervised_classifier_defs('detailed')
classifiers = {model_name : model_instances[model_name] for model_name in model_names}
param_grid = {model_name : params[model_name] for model_name in model_names}

We can see below what the `param_grid` look like:

In [12]:
for model_name, params in param_grid.items():
    print(model_name)
    for param_name, param_vals in params.items():
        print(f'{param_name}: {param_vals}')
    print()

LinearSVC
clf__C: [0.01, 0.1, 1, 10.0, 100.0, 1000.0]
clf__max_iter: [800, 1000, 1200, 1500, 2000]

LogisticRegression
clf__solver: ['newton-cg', 'lbfgs', 'sag']
clf__C: [1.00000000e+00 2.78255940e+00 7.74263683e+00 2.15443469e+01
 5.99484250e+01 1.66810054e+02 4.64158883e+02 1.29154967e+03
 3.59381366e+03 1.00000000e+04]

PassiveAggressiveClassifier
clf__C: [0.001, 0.01, 0.1, 1.0]
clf__loss: ['hinge', 'squared_hinge']



Finally, all we need to do is pass everything into `classification_deployer()`:

In [13]:
summary_table, best_model = classification_deployer(x,
                                                    y,
                                                    n_splits=5,
                                                    classifiers=classifiers,
                                                    search_space=param_grid,
                                                    pipeline_steps=pipeline_steps,
                                                    scoring=scorer)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\Users\agmoore\AppData\Local\anaconda3\envs\pvops_bare_with_tweaks\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Better score (0.835) found on classifier: LinearSVC
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Better score (0.847) found on classifier: LogisticRegression
Fitting 5 folds for each of 8 candidates, totalling 40 fits


And below, we can see a summary of how the models performed.

In [14]:
sorted_table = summary_table.sort_values('mean_score').iloc[::-1].head(10)
sorted_table

,clf__C,clf__max_iter,mean_fit_time,estimator,min_score,max_score,mean_score,std_score,clf__solver,clf__loss
40,21.544347,NaN,0.049305,LogisticRegression,0.705882,0.941176,0.847059,0.088039,lbfgs,NaN
39,21.544347,NaN,0.057756,LogisticRegression,0.705882,0.941176,0.847059,0.088039,newton-cg,NaN
38,7.742637,NaN,0.043245,LogisticRegression,0.705882,0.941176,0.847059,0.088039,sag,NaN
37,7.742637,NaN,0.039319,LogisticRegression,0.705882,0.941176,0.847059,0.088039,lbfgs,NaN
36,7.742637,NaN,0.061179,LogisticRegression,0.705882,0.941176,0.847059,0.088039,newton-cg,NaN
22,100.0,1200,0.060737,LinearSVC,0.705882,0.941176,0.835294,0.094118,NaN,NaN
26,1000.0,1000,0.057189,LinearSVC,0.705882,0.941176,0.835294,0.094118,NaN,NaN
52,1291.549665,NaN,0.050588,LogisticRegression,0.705882,0.941176,0.835294,0.094118,lbfgs,NaN
66,1.0,NaN,0.045288,PassiveAggressiveClassifier,0.705882,0.941176,0.835294,0.094118,NaN,hinge
67,1.0,NaN,0.025799,PassiveAggressiveClassifier,0.705882,0.941176,0.823529,0.105227,NaN,squared_hinge


Due to the small example dataset, many of the models have the same performance. Still, we can see `LogisticRegression` generally performed the best. Also, note that our best-performing models did perform notably better than our naive approach, so we do gain something by using machine learning methods.

`classification_deployer()` only returns the best model, but we can easily refit using just the parameters we want.

In [15]:
# get the row for the model we want to use
model_row = sorted_table.iloc[1,:]

# pull out the model name and parameters
model_name = model_row['estimator']
model_params = model_row[list(param_grid[model_name].keys())].to_dict()

# parameter values have to be in a list, even if there is just one
model_params = {param_name: [param_val] for param_name, param_val in model_params.items()}

# get into the form used by classification_deployer()
single_classifier = {model_name : model_instances[model_name]}
single_param_grid = {model_name : model_params}
single_classifier, single_param_grid

({'LogisticRegression': LogisticRegression()},
 {'LogisticRegression': {'clf__solver': ['newton-cg'],
   'clf__C': [21.544346900318832]}})

Now that we have what we need, we can quickly rerun the fit and save the final model.

In [16]:
_, final_model = classification_deployer(x, y,
                                            n_splits=5,
                                            classifiers=single_classifier,
                                            search_space=single_param_grid,
                                            pipeline_steps=pipeline_steps,
                                            scoring=scorer)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Better score (0.847) found on classifier: LogisticRegression


### Gap-filling the asset using our final model

Now, we can use our model to make predictions on the rows with a missing `Asset` (but with a `GeneralDesc`). First, we can pull out the data we need to feed into the model.

In [17]:
records_to_gapfill = CM_df[CM_df['Asset'].isnull()
                           & ~CM_df['CompletionDesc'].isnull()].copy()
# only need to keep around the CompletionDesc field
records_to_gapfill = records_to_gapfill['CompletionDesc']

Now, we follow the same cleaning procedure as before, and finally use our model to predict the asset.

In [18]:
records_to_gapfill = records_to_gapfill.apply(lambda x: text_remove_numbers_stopwords(x, stopwords))
records_to_gapfill = records_to_gapfill.str.lower()
pred_is_inverter = final_model.predict(records_to_gapfill)

Now, we can use our prediction as a mask to pull out a list of indices where the asset is predicted to be an inverter.

In [19]:
pred_inverter_indices = records_to_gapfill.index[pred_is_inverter.astype(bool)]
pred_inverter_indices

Index([5, 6, 31, 35, 77], dtype='int64')

We can also construct a mask for the datapoints without null values for the asset. By combining these two masks, we get a final set of inverter-related records that can be used for other analysis.

In [20]:
inverter_indices = CM_df.index[~CM_df['Asset'].isnull()
                               & CM_df['Asset'].str.contains('Inverter')]

inverter_CM_df = CM_df.loc[list(inverter_indices) + list(pred_inverter_indices), :]
inverter_CM_df.loc[pred_inverter_indices, 'Asset'] = 'Inverter'
inverter_CM_df

,Unnamed: 0,Date_EventStart,Date_EventEnd,Asset,CompletionDesc,Cause,ImpactLevel,randid
3,3,11/14/2017 7:46,11/14/2017 8:35,Inverter,14 nov: we were alerted that e-c3-1 had faulte...,019 - Unplanned outage/derate.,Underperformance,54
12,12,7/12/2017 17:01,8/15/2017 16:04,Inverter,self-resolved. techdispatched: no,019 - Unplanned outage/derate.,Underperformance,11
13,13,8/15/2017 12:01,8/16/2017 8:00,Inverter,we reset communications inverter reset switch ...,020 - Data/Communication System Outage.,Underperformance,55
16,16,1/18/2018 13:49,1/25/2018 11:30,Inverter,the inverter was taken offline transformer shu...,019 - Unplanned outage/derate.,Underperformance,24
25,25,12/7/2019 11:55,12/7/2019 17:00,Inverter,c4 performed remote hard reset to faults inver...,019 - Unplanned outage/derate,Underperformance,39
36,36,6/8/2019 8:45,6/9/2019 11:35,Inverter,6/9/19 - inverter 3 was offline with ground fa...,001 - Ground fault,Underperformance,16
38,38,6/21/2017 17:25,6/23/2017 12:00,Inverter,confirmed that cb 1.1.6 was turned off. verifi...,019 - Unplanned outage/derate.,Underperformance,34
41,41,5/28/2019 15:01,6/18/2019 17:00,Inverter,pcb master core were swapped. inverter has bee...,019 - Unplanned outage/derate,Underperformance,18
43,43,3/23/2017 16:06,3/24/2017 9:58,Inverter,investigate inverter down. inverter 3.5 had re...,.,Underperformance,29
49,49,3/3/2018 13:44,3/3/2018 15:38,Inverter,[no completion notes were entered servicemax]....,019 - Unplanned outage/derate.,Underperformance,14


In [21]:
inverter_CM_df.to_csv('example_data/example_ML_ticket_data_gapfilled_inverter_records.csv', index=False)